In [ ]:
import sys, os; sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__) if '__file__' in globals() else os.getcwd(), '..')))
#import os; os.chdir(os.path.dirname(os.getcwd()))
from utils.model_loader import get_model_fits
import numpy as np
import pandas as pd
import re
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
import numpy as np
from scipy.stats import norm
from scipy.special import logsumexp
data_dir = f"datasets/friedman"
results_dir_relu = "results/regression/single_layer/relu/friedman"
results_dir_tanh = "results/regression/single_layer/tanh/friedman"

model_names_relu = ["Gaussian", "Regularized Horseshoe", "Dirichlet Horseshoe", "Dirichlet Student T"]#, "Pred CP"]
model_names_tanh = ["Gaussian tanh", "Regularized Horseshoe tanh", "Dirichlet Horseshoe tanh", "Dirichlet Student T tanh"]#, "Pred CP tanh"]


relu_fits = {}
tanh_fits = {}

files = sorted(f for f in os.listdir(data_dir) if f.endswith(".npz"))
for fname in files:
    base_config_name = fname.replace(".npz", "")  # e.g., "GAM_N100_p8_sigma1.00_seed1"
    full_config_path = f"{base_config_name}"  # → "type_1/GAM_N100_p8_sigma1.00_seed1"
    relu_fit = get_model_fits(
        config=full_config_path,
        results_dir=results_dir_relu,
        models=model_names_relu,
        include_prior=False,
    )
    
    tanh_fit = get_model_fits(
        config=full_config_path,
        results_dir=results_dir_tanh,
        models=model_names_tanh,
        include_prior=False,
    )
    

    relu_fits[base_config_name] = relu_fit  # use clean key
    tanh_fits[base_config_name] = tanh_fit  # use clean key
    


In [4]:
dataset_key = f'Friedman_N{100}_p10_sigma{1:.2f}_seed{1}'
path = f"datasets/friedman/{dataset_key}.npz"

data = np.load(path)
X = data["X_train"]
Y = data["y_train"]#.squeeze()  # shape (N_test,)

In [25]:
# Training data
X_train = X          # shape (n_train, d)
y_train = Y          # shape (n_train,)

X_test = data["X_test"]
y_test = data["y_test"]#.squeeze()  # shape (N_test,)

# Posterior draws of f(x_train)
# Shape: (T, n_train), same as your func_samples example
fit = tanh_fits['Friedman_N100_p10_sigma1.00_seed1']['Dirichlet Horseshoe tanh']['posterior']
f_train_samples = fit.stan_variable("output").squeeze(-1)
f_test_samples = fit.stan_variable("output_test").squeeze(-1)

# OPTIONAL: posterior draws of noise std dev, shape (T,)
# If you don't have this, we make a rough estimate later.
sigma_samples = fit.stan_variable("sigma")    # or np.array([...]) of shape (T,)


In [26]:
def compute_aoi_weights(y_new, f_new_samples, sigma_samples):
    """
    AOI importance weights for a single candidate y_new and one test point.
    
    f_new_samples: shape (T,)
    sigma_samples: shape (T,)
    
    Returns: weights shape (T,) summing to 1 (in log-space for stability).
    """
    # log p(y_new | f_new_samples, sigma)
    loglik_new = norm.logpdf(y_new, loc=f_new_samples, scale=sigma_samples)  # shape (T,)
    log_w = loglik_new - logsumexp(loglik_new)
    w = np.exp(log_w)
    return w  # shape (T,)

def predictive_density_augmented(y_train, f_train_samples, y_new, f_new_samples, sigma_samples):
    """
    Compute conformity scores (predictive density) for:
      - each training point i = 0..n-1
      - the new point n (the candidate y_new)
    
    using AOI weights for the augmented dataset.
    
    Returns:
      scores_train: shape (n,)
      score_new: scalar
    """
    T, n = f_train_samples.shape
    
    # AOI weights for this candidate y_new
    w = compute_aoi_weights(y_new, f_new_samples, sigma_samples)  # shape (T,)
    
    # likelihood for all training points under each draw: p(y_i | f^{(t)}(x_i), sigma_t)
    # shape (T, n)
    train_lik = norm.pdf(
        y_train[None, :], 
        loc=f_train_samples, 
        scale=sigma_samples[:, None]
    )
    
    # predictive density under augmented posterior: sum_t w_t * lik_t
    scores_train = np.sum(w[:, None] * train_lik, axis=0)  # shape (n,)
    
    # same for the new point (candidate y_new)
    new_lik = norm.pdf(
        y_new,
        loc=f_new_samples,
        scale=sigma_samples
    )  # shape (T,)
    score_new = np.sum(w * new_lik)
    
    return scores_train, score_new

def cb_value_for_y_candidate(
    y_new, 
    y_train, 
    f_train_samples, 
    f_new_samples, 
    sigma_samples
):
    """
    Compute the CB value π(y_new) for a single test point and candidate y_new.
    """
    scores_train, score_new = predictive_density_augmented(
        y_train, f_train_samples, y_new, f_new_samples, sigma_samples
    )
    
    # combine scores: n training + 1 new
    all_scores = np.concatenate([scores_train, np.array([score_new])])
    
    # rank-based p-value (ties are negligible in continuous case)
    cb_val = np.mean(all_scores <= score_new)
    return cb_val

def cb_interval_for_one_point(
    j,
    y_train,
    f_train_samples,
    f_test_samples,
    sigma_samples,
    y_grid,
    alpha=0.1,
):
    """
    Compute a CB interval for test point j at level 1-alpha using a candidate grid y_grid.
    
    Returns:
      cb_vals: shape (len(y_grid),) with π(y_grid[k])
      interval_mask: boolean mask where π(y) > alpha
      (y_min, y_max): endpoints of the CB interval on the grid, or (None, None) if empty
    """
    T, n_train = f_train_samples.shape
    _, n_test = f_test_samples.shape
    
    assert 0 <= j < n_test
    
    # function samples for this test point
    f_new_samples = f_test_samples[:, j]  # shape (T,)
    
    cb_vals = np.zeros_like(y_grid, dtype=float)
    
    for k, y_new in enumerate(y_grid):
        cb_vals[k] = cb_value_for_y_candidate(
            y_new=y_new,
            y_train=y_train,
            f_train_samples=f_train_samples,
            f_new_samples=f_new_samples,
            sigma_samples=sigma_samples
        )
    
    interval_mask = cb_vals > alpha
    if not np.any(interval_mask):
        return cb_vals, interval_mask, (None, None)
    
    # Take min and max y where π(y) > alpha
    y_in = y_grid[interval_mask]
    return cb_vals, interval_mask, (float(y_in.min()), float(y_in.max()))


In [27]:
# 1. Get sigma_samples
#sigma_samples = get_sigma_samples(f_train_samples, y_train, sigma_samples)

# 2. Define a y-grid (e.g. based on posterior predictive range)
#    For each test point you might center it differently, but here is one global example:
f_all = np.concatenate([f_train_samples, f_test_samples], axis=1)  # shape (T, n_train + n_test)
f_mean_all = f_all.mean(axis=0)
f_sd_all   = f_all.std(axis=0)

y_min = f_mean_all.min() - 3 * f_sd_all.max()
y_max = f_mean_all.max() + 3 * f_sd_all.max()
y_grid = np.linspace(y_min, y_max, 200)  # 200 grid points

alpha = 0.2  # e.g. 80% CB interval

n_test = f_test_samples.shape[1]
cb_intervals = []

for j in range(n_test):
    cb_vals_j, mask_j, (lo_j, hi_j) = cb_interval_for_one_point(
        j=j,
        y_train=y_train,
        f_train_samples=f_train_samples,
        f_test_samples=f_test_samples,
        sigma_samples=sigma_samples,
        y_grid=y_grid,
        alpha=alpha,
    )
    cb_intervals.append((lo_j, hi_j))

cb_intervals = np.array(cb_intervals)  # shape (n_test, 2)


In [36]:
intervals = {}

def cp_runner(fits, dataset_key, y_train):
    intervals = {}
    for fit in fits:
        f_train_samples = tanh_fits[dataset_key][fit]['posterior'].stan_variable("output").squeeze(-1)
        f_test_samples = tanh_fits[dataset_key][fit]['posterior'].stan_variable("output_test").squeeze(-1)
        sigma_samples = tanh_fits[dataset_key][fit]['posterior'].stan_variable("sigma")
        f_all = np.concatenate([f_train_samples, f_test_samples], axis=1)  # shape (T, n_train + n_test)
        f_mean_all = f_all.mean(axis=0)
        f_sd_all   = f_all.std(axis=0)

        y_min = f_mean_all.min() - 3 * f_sd_all.max()
        y_max = f_mean_all.max() + 3 * f_sd_all.max()
        y_grid = np.linspace(y_min, y_max, 200)  # 200 grid points

        alpha = 0.2  # e.g. 80% CB interval

        n_test = f_test_samples.shape[1]
        cb_intervals = []

        for j in range(n_test):
            cb_vals_j, mask_j, (lo_j, hi_j) = cb_interval_for_one_point(
                j=j,
                y_train=y_train,
                f_train_samples=f_train_samples,
                f_test_samples=f_test_samples,
                sigma_samples=sigma_samples,
                y_grid=y_grid,
                alpha=alpha,
            )
            cb_intervals.append((lo_j, hi_j))

        intervals[fit] = np.array(cb_intervals)
    return intervals 

In [37]:
dataset_key = f'Friedman_N{500}_p10_sigma{1:.2f}_seed{11}'
path = f"datasets/friedman/{dataset_key}.npz"

data = np.load(path)
X_train = data["X_train"]
y_train = data["y_train"]

X_test = data["X_test"]
y_test = data["y_test"]

fits = ["Gaussian tanh", "Regularized Horseshoe tanh", "Dirichlet Horseshoe tanh", "Dirichlet Student T tanh"]#, "Pred CP tanh"]

ints = cp_runner(fits, dataset_key=dataset_key, y_train=y_train)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# intervals: your dict from before
# intervals = {
#     "Gaussian tanh": ...,
#     "Regularized Horseshoe tanh": ...
# }

models = list(ints.keys())
n_obs = ints[models[0]].shape[0]
x = np.arange(n_obs)

fig, ax = plt.subplots(figsize=(8, 5))

# small horizontal offsets so intervals don't sit exactly on top of each other
offsets = np.linspace(-0.1, 0.1, len(models))

for off, model in zip(offsets, models):
    intv = ints[model]           # shape (n_obs, 2)
    lows = intv[:, 0]
    highs = intv[:, 1]
    mids = 0.5 * (lows + highs)
    yerr = np.vstack([mids - lows, highs - mids])

    ax.errorbar(
        x + off,
        mids,
        yerr=yerr,
        fmt="o",
        capsize=3,
        label=model,
    )

ax.set_xlabel("Observation index")
ax.set_ylabel("Response")
ax.set_title("Credible intervals per observation for each model")
ax.legend()
plt.tight_layout()
plt.show()
